In [2]:
import ocr
from tqdm import tqdm
import re
import pandas as pd

In [3]:
circular_df = pd.read_csv("../output_files/2023_2024_policy_circulars.csv")
len(circular_df)

234

Delete rows where the words are found in `downloaded_png_path`

In [4]:
mask = circular_df['title'].str.lower().apply(
    lambda x: all(word.lower() not in re.split(r"[ \-_]+", x) for word in ['vol', 'volume', 'meeting', 'Report', 'Act', 'A4', 'Publication', 'half-year', 'bullion', 'calendar', 
    'qtr', 'annual', 'quarter', 'remark', 'speech', 'letter', 'ECR', 'Analysis', 'weekly'])
)
circular_df = circular_df[mask]

print(circular_df.shape)

(189, 10)


In [5]:
circular_df.head(2)

,_id,pageNumber,reference,document,title,timestamp,__v,year,downloaded_pdf_path,downloaded_png_path
0,6623ba0dfaf3e17e8b6016d6,1,TED/DIR/CON/GOM/001/073,https://cbn.gov.ng/Out/2024/CCD/Sales of FX to...,Sales of FX to BDCs to Meet Retail Market Dema...,2024-04-08,0,2024,downloaded_pdfs/Sales of FX to BDCs to Meet Re...,png_files/Sales of FX to BDCs to Meet Retail M...
2,6623ba0dfaf3e17e8b6016d8,1,FPR/DIR/PUB/CIR/002/009,https://cbn.gov.ng/Out/2024/CCD/Recapitalizati...,Review of Minimum Capital Requirements for Com...,2024-03-28,0,2024,downloaded_pdfs/Recapitalization_MARCH_2024.pdf,NaN


In [6]:
## use rows from October 2023, representing Cardoso's tenure

circular_df['timestamp'] = pd.to_datetime(circular_df['timestamp'], format='%Y-%m-%d')

# Filter rows for October 2023
circular_df = circular_df[circular_df['timestamp'] >= pd.Timestamp(year=2023, month=10, day=1)]

# Print the resulting DataFrame
print(len(circular_df))

76


In [7]:
circular_df.isna().sum()

_id                    0
pageNumber             0
reference              0
document               0
title                  0
timestamp              0
__v                    0
year                   0
downloaded_pdf_path    0
downloaded_png_path    8
dtype: int64

In [8]:
circular_df = circular_df.dropna()
len(circular_df)

68

In [10]:
recognize = ocr.TextRecognizer()

def process_image(path):
    try:
        return recognize.recognize_img(path)
    except Exception as e:
        print(f"Error processing image {path}: {str(e)}")
        return None

# apply OCR to each image in the df, using parallel processing if necessary
extracted_texts = []
for img_path in tqdm(circular_df["downloaded_png_path"]):
    text = process_image("../output_files/"+img_path)
    extracted_texts.append(text)


100%|██████████| 68/68 [25:03<00:00, 22.11s/it]


In [12]:
circular_df["text"] = extracted_texts

In [13]:
circular_df.head()

,_id,pageNumber,reference,document,title,timestamp,__v,year,downloaded_pdf_path,downloaded_png_path,text
0,6623ba0dfaf3e17e8b6016d6,1,TED/DIR/CON/GOM/001/073,https://cbn.gov.ng/Out/2024/CCD/Sales of FX to...,Sales of FX to BDCs to Meet Retail Market Dema...,2024-04-08,0,2024,downloaded_pdfs/Sales of FX to BDCs to Meet Re...,png_files/Sales of FX to BDCs to Meet Retail M...,"{Internal server error: , (1, 'Image too large..."
3,6623ba0dfaf3e17e8b6016dd,1,CCD/FAQ/01/16-03-2024,https://cbn.gov.ng/Out/2024/CCD/FAQs for ferti...,Partnership to Tackle Food Inflation: Fertili...,2024-03-16,0,2024,downloaded_pdfs/FAQs for fertilisers02.pdf,png_files/FAQs for fertilisers02.png,PARTNERSHIP TO TACKLE FOOD INFLATION:\nFERTILI...
4,6623ba0dfaf3e17e8b6016de,1,BSD/DIR/PUB/LAB/017/003,https://cbn.gov.ng/Out/2024/CCD/RE IMPACT OF R...,Re: Impact of Recent Policy Reforms-Prudential...,2024-03-14,0,2024,downloaded_pdfs/RE IMPACT OF RECENT POLICY REF...,png_files/RE IMPACT OF RECENT POLICY REFORMS-P...,Central Bank of Nigeria\n\nBanking Supervision...
5,6623ba0dfaf3e17e8b6016df,1,CCD\SP\GEN\01\14-03-2024,https://cbn.gov.ng/Out/2024/CCD/Governor's Rem...,Handover of Fertilizer to the Ministry of Agri...,2024-03-14,0,2024,downloaded_pdfs/Governor's Remarks At Handover...,png_files/Governor's Remarks At Handover of Fe...,HANDOVER OF FERTILIZER TO THE MINISTRY OF AGRI...
6,6623ba0dfaf3e17e8b6016e0,1,FMD/DIR/PUB/CIR/001/010,https://cbn.gov.ng/Out/2024/FMD/Superceded Cir...,Superseded Circulars/Guidelines (REVIEWED),2024-03-12,0,2024,downloaded_pdfs/Superceded Circulars and Guide...,png_files/Superceded Circulars and Guidelines ...,Central Bank of Nigeria\n\nFinancial Markets D...


In [17]:
circular_df["text"] = circular_df["text"].astype(str)

In [18]:
filtered_cicular_df = circular_df[circular_df["text"]!="{'Internal server error: ', TesseractError(1, 'Image too large: (1654, 84205) Error during processing.')}"]
len(filtered_cicular_df)

67

In [20]:
filtered_cicular_df['timestamp'] = pd.to_datetime(filtered_cicular_df['timestamp'])
filtered_cicular_df['timestamp'] = filtered_cicular_df['timestamp'].astype(str)

/var/folders/2x/3p0r8mnx6s7f7rwwg4fjmv3c0000gn/T/ipykernel_27266/2533319880.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_cicular_df['timestamp'] = pd.to_datetime(filtered_cicular_df['timestamp'])
/var/folders/2x/3p0r8mnx6s7f7rwwg4fjmv3c0000gn/T/ipykernel_27266/2533319880.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_cicular_df['timestamp'] = filtered_cicular_df['timestamp'].astype(str)


In [22]:
filtered_cicular_df.to_parquet("../output_files/text_extracted_circular_df_oct2023toApril2024.parquet", index=False)